# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 4:17PM,237697,10,SO202201394,Yusen – 3D Matrix,Released
1,May 19 2022 4:17PM,237696,10,8041612,"Herbivore Botanicals, LLC.",Released
2,May 19 2022 4:17PM,237696,10,8041636,"Herbivore Botanicals, LLC.",Released
3,May 19 2022 4:17PM,237696,10,8041640,"Herbivore Botanicals, LLC.",Released
4,May 19 2022 4:09PM,237695,10,Enova-9994,Emerginnova,Released
5,May 19 2022 4:09PM,237695,10,Enova-9995,Emerginnova,Released
6,May 19 2022 4:07PM,237694,10,6482857,Acute Outpatient Solutions,Released
7,May 19 2022 4:00PM,237693,10,Stone8041619,Acute Outpatient Solutions,Released
8,May 19 2022 3:58PM,237692,16,SSF-8041615,Sartorius Stedim Filters Inc.,Released
9,May 19 2022 3:35PM,237690,19,8041610,"Emersa Waterbox, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,237693,Released,1
34,237694,Released,1
35,237695,Released,2
36,237696,Released,3
37,237697,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237693,NaN,NaN,1.0
237694,NaN,NaN,1.0
237695,NaN,NaN,2.0
237696,NaN,NaN,3.0
237697,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,0.0,3.0,2.0
237596,0.0,0.0,1.0
237597,0.0,0.0,1.0
237598,0.0,0.0,1.0
237601,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,0,3,2
237596,0,0,1
237597,0,0,1
237598,0,0,1
237601,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,0,3,2
1,237596,0,0,1
2,237597,0,0,1
3,237598,0,0,1
4,237601,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,,3,2
1,237596,,,1
2,237597,,,1
3,237598,,,1
4,237601,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix
1,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC."
4,May 19 2022 4:09PM,237695,10,Emerginnova
6,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions
7,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions
8,May 19 2022 3:58PM,237692,16,Sartorius Stedim Filters Inc.
9,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC"
10,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc."
11,May 19 2022 3:29PM,237688,19,"Emersa Waterbox, LLC"
12,May 19 2022 2:56PM,237683,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,,,1
1,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",,,3
2,May 19 2022 4:09PM,237695,10,Emerginnova,,,2
3,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,,,1
4,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions,,,1
5,May 19 2022 3:58PM,237692,16,Sartorius Stedim Filters Inc.,,,1
6,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC",,,1
7,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc.",,,1
8,May 19 2022 3:29PM,237688,19,"Emersa Waterbox, LLC",,,1
9,May 19 2022 2:56PM,237683,10,ISDIN Corporation,,,47


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,1,,
1,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",3,,
2,May 19 2022 4:09PM,237695,10,Emerginnova,2,,
3,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,1,,
4,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions,1,,
5,May 19 2022 3:58PM,237692,16,Sartorius Stedim Filters Inc.,1,,
6,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC",1,,
7,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc.",1,,
8,May 19 2022 3:29PM,237688,19,"Emersa Waterbox, LLC",1,,
9,May 19 2022 2:56PM,237683,10,ISDIN Corporation,47,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,1,,
1,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",3,,
2,May 19 2022 4:09PM,237695,10,Emerginnova,2,,
3,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,1,,
4,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",3.0,NaN,NaN
2,May 19 2022 4:09PM,237695,10,Emerginnova,2.0,NaN,NaN
3,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,1.0,NaN,NaN
4,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",3.0,0.0,0.0
2,May 19 2022 4:09PM,237695,10,Emerginnova,2.0,0.0,0.0
3,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,1.0,0.0,0.0
4,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2614321,88.0,34.0,8.0
12,237676,1.0,0.0,0.0
16,1425911,5.0,1.0,0.0
17,237654,0.0,1.0,0.0
18,950434,4.0,0.0,0.0
19,950696,3.0,1.0,0.0
20,712874,22.0,3.0,0.0
22,475281,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2614321,88.0,34.0,8.0
1,12,237676,1.0,0.0,0.0
2,16,1425911,5.0,1.0,0.0
3,17,237654,0.0,1.0,0.0
4,18,950434,4.0,0.0,0.0
5,19,950696,3.0,1.0,0.0
6,20,712874,22.0,3.0,0.0
7,22,475281,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,88.0,34.0,8.0
1,12,1.0,0.0,0.0
2,16,5.0,1.0,0.0
3,17,0.0,1.0,0.0
4,18,4.0,0.0,0.0
5,19,3.0,1.0,0.0
6,20,22.0,3.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,12,Released,1.0
2,16,Released,5.0
3,17,Released,0.0
4,18,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,22
Status,,,,,,,,
Completed,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,34.0,0.0,1.0,1.0,0.0,1.0,3.0,0.0
Released,88.0,1.0,5.0,0.0,4.0,3.0,22.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,22
0,Completed,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,0.0,1.0,1.0,0.0,1.0,3.0,0.0
2,Released,88.0,1.0,5.0,0.0,4.0,3.0,22.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,22
0,Completed,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,0.0,1.0,1.0,0.0,1.0,3.0,0.0
2,Released,88.0,1.0,5.0,0.0,4.0,3.0,22.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()